In [2]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from evaluate import load
import ast
import json



In [3]:
data_v10_interpolation = pd.read_csv("./interpolation-diffusion-rocstories-16-d=5-v1.0-t=0.1.csv")
data_v11_interpolation = pd.read_csv("./interpolation-diffusion-rocstories-16-d=5-v1.1-t=0.1.csv")

In [13]:
for ind in [0, 25, 50, 75, 99]:
    texts = data_v11_interpolation.iloc[ind]["pred_text"]
    texts = ast.literal_eval(texts)
    print(texts[2])

Kay was drying clothes when she heard the washer stop. She assumed the load finished and went to change it to the washer. She was confused to see water in the washer. She tried to turn it on but the washer wouldn't turn back on. Kay had to wring her clothes out by hand to finish her laundry.
Kay was drying clothes when she heard the washer stop. She assumed the load finished and went to change it to the washer. She was surprised to see water in the washer. She tried to turn it on but the washer wouldn't come back on. Kay had to war her clothes out by hand to finish her laundry.
Ana was sleeping things sleep when reached her washer'door. She woke was in and went the dream. had to had to a dryer scared she saw her mom in it. But Ana didn to turn it to or her washer dcuray it on on. Ana kept to wrow her sleep back asleep. to finish her laundry.
Ana was at a sleepover at her friend's house. She woke in the middle of the night. She'd had a bad dream and she wished her mom was there! But Ana

In [14]:
model_id = 'gpt2-large'
perplexity = load("perplexity", module_type="metric", model_id=model_id)


In [15]:
texts = data_v11_interpolation.iloc[50]["pred_text"]
texts = ast.literal_eval(texts)

ppl = perplexity.compute(
    predictions=texts, 
    model_id=model_id, 
    device='cuda', 
    add_start_token=True,
)["perplexities"]

  0%|          | 0/7 [00:00<?, ?it/s]

In [17]:
np.mean(ppl)

280.6277895736694

In [19]:
np.argsort(ppl)

array([49,  4, 30, 75, 53, 10, 92, 69, 14, 27, 86, 21, 48, 38, 78, 52, 68,
       88, 85, 93, 67,  3, 25, 36, 41, 83, 18, 29, 73, 80, 99, 13, 70, 19,
       31,  9, 63, 46, 26, 97,  2, 47, 79, 56, 40, 64, 72,  7, 33, 91,  0,
       57, 23, 84, 43, 15, 71, 74, 11, 90, 61, 32, 62,  1, 51, 45, 76, 77,
       60, 58, 35, 12, 16, 44, 95, 87, 96, 24,  6,  8, 59, 81, 22, 98, 54,
       39, 37, 89, 94, 82, 28,  5, 50, 20, 17, 66, 55, 65, 34, 42])

In [29]:
text_ind = 49
for ind in [0, 40, 50, 60, 99]:
    texts = data_v11_interpolation.iloc[ind]["pred_text"]
    texts = ast.literal_eval(texts)
    print(texts[text_ind])

Stewart invited me to his birthday party. I wasn't that good of friends with him so I couldn't find a gift. I texteded his girlfriend but she wasn't replying! Eventually I purchased a $ 25 gift card. He loved the gift so much and said it was the best one ever.
Stewart invited my back her birthday party! I wasn't that good of a with her so I couldn't find a gift. I emailed her, but she wasn't replying! Eventually I purchased a $ 25 gift card. He loved the gift so much and said it was the best one ever.
Kelly invited me got her birthday party for her birthday. Kelly thought it was a sweet one. I quickly put no put it on. When she tried it, she was disgusted. It was horrible!
Kelly's boyfriend got her perfume for her birthday. Kelly thought it was a sweet gift. She immediately put some of it on. When she tried it, she was disgusted. It was horrible.
Kelly's boyfriend got her perfume for her birthday. Kelly thought it was a sweet gesture. She immediately put some of it on. When she tried i

In [4]:
data = json.load(open("interpolation-all_times-diffusion-rocstories-16-d=5-v2.3.4.json", "r"))

In [5]:
data.keys()

dict_keys(['0.0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '1.0'])

In [6]:
data["0.0"].keys()

dict_keys(['alpha', 'pred_text', 'x_t_texts'])

In [7]:
len(data["0.0"]["pred_text"])

100

In [8]:
t = 50

In [9]:
len(data["0.0"]["pred_text"][t])

1024

In [12]:
ppl_data = json.load(open("interpolation-all_times-diffusion-rocstories-16-d=5-v2.3.4-ppl.json", "r"))

In [19]:
np.argsort(ppl_data["0.0"][t])[::-1][:10]


array([376, 727,   8,  94, 382,  26, 231, 489, 659, 777])

In [23]:
ind = 94
text_1 = data["0.0"]["pred_text"][0][ind]
text_m = data["0.0"]["pred_text"][t][ind]
text_2 = data["0.0"]["pred_text"][-1][ind]

print(f"text_1: {text_1}")
print(f"text_m: {text_m}")
print(f"text_2: {text_2}")


text_1: Kurt graduated from law school ten years ago. His school held a ten - year class reunion. At the reunion, Kurt bragged about how many Twitter followers he had. The other attorneys at the reunion laughed behind his back. Kurt did not realize how shallow and stupid he looked at the reunion.
text_m: Kurt graduated up law school ten school ago. needed to hold on ten eye year class reunion. At kind of sore Kurt bragged about many times Twitter't had on the other makeup. Instead reunion decided to go back school without it. realize ended shallow loving stupid he adopted it as the permanent look.
text_2: Tammy woke up one morning before school. She needed to put on her eye makeup but she had some type of sore. She tried several times but couldn't put on her eye makeup. Instead she decided to go to school without it. She ended up loving it and adopted it as her permanent look.


In [ ]:
ppl_